In [ ]:
!pip install spotipy


     |████████████████████████████████| 62 kB 798 kB/s 
     |████████████████████████████████| 138 kB 39.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '8bc466d7fa994c40b1c0ad5ecde205ad' 
secret = 'b045478038744d0a9a1917e2be70e15d'


client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
spotify_api = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

print(spotify_api)

In [ ]:
import json
import pandas as pd

playlist_features_list = ["artist", "album", "track_name", "track_id", "release_date", "explicit", "popularity",
                             "danceability", "energy", "key", "loudness", "mode", "speechiness",
                             "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
                             
def analyze_playlist (username, playlist_id):

  offset = 0
  playlist_df = pd.DataFrame(columns = playlist_features_list)
  playlist = spotify_api.user_playlist_tracks(username, playlist_id, limit = 100, offset=offset)["items"]
  count = 0

  #print(playlist[0]["track"]["album"]["artists"][0]["name"])

  for track in playlist:
    playlist_data = {}
    playlist_data["artist"] = track["track"]["album"]["artists"][0]["name"]
    playlist_data["album"] = track["track"]["album"]["name"]
    playlist_data["track_name"] = track["track"]["name"]
    playlist_data["track_id"] = track["track"]["id"]
    playlist_data["release_date"] = track["track"]["album"]["release_date"]
    playlist_data["explicit"] = track["track"]["explicit"]
    playlist_data["popularity"] = track["track"]["popularity"]
    
    audio_data = spotify_api.audio_features(playlist_data["track_id"])[0]
    for data in playlist_features_list[7:]:
      playlist_data[data] = audio_data[data]
    
    track_df = pd.DataFrame(playlist_data, index = [0])
    playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
    count += 1
  
  #topRange * 100 is the amount of songs that will be scraped per playlist
  topRange = 100
  for i in range(1,topRange):
    offset = 100 * i
    playlist = spotify_api.user_playlist_tracks(username, playlist_id, limit = 100, offset=offset)["items"]
    #print(playlist[0]["track"]["album"]["artists"][0]["name"] + " " + str(count))
    
    for track in playlist:
      if(track["track"] is not None):
        try:
          playlist_data = {}
          playlist_data["artist"] = track["track"]["album"]["artists"][0]["name"]
          playlist_data["album"] = track["track"]["album"]["name"]
          playlist_data["track_name"] = track["track"]["name"]
          playlist_data["track_id"] = track["track"]["id"]
          playlist_data["release_date"] = track["track"]["album"]["release_date"]
          playlist_data["explicit"] = track["track"]["explicit"]
          playlist_data["popularity"] = track["track"]["popularity"]
    
          audio_data = spotify_api.audio_features(playlist_data["track_id"])[0]
          for data in playlist_features_list[7:]:
            playlist_data[data] = audio_data[data]
    
          track_df = pd.DataFrame(playlist_data, index = [0])
          playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
          count += 1
        except:
          print("     exception")
          print(json.dumps(track, indent=4))

  return list(playlist_df.T.to_dict().values())

In [ ]:
import json
#print(json.dumps(analyze_playlist("mfaria2020" , "6dgZo2bQOd1ggw8Ac0nFIa"), indent=4))
print(len(analyze_playlist("DuckLover/F4F/Fox&WolfTeamAnim" , "1151nuJMpCvYjgPaN1HCRk")))

     exception
{
    "added_at": "2021-11-27T11:13:33Z",
    "added_by": {
        "external_urls": {
            "spotify": "https://open.spotify.com/user/gzgq61tpbqxgw6u68p8xqnu8f"
        },
        "href": "https://api.spotify.com/v1/users/gzgq61tpbqxgw6u68p8xqnu8f",
        "id": "gzgq61tpbqxgw6u68p8xqnu8f",
        "type": "user",
        "uri": "spotify:user:gzgq61tpbqxgw6u68p8xqnu8f"
    },
    "is_local": false,
    "primary_color": null,
    "track": {
        "album": {
            "album_type": "single",
            "artists": [
                {
                    "external_urls": {
                        "spotify": "https://open.spotify.com/artist/1Cs0zKBU1kc0i8ypK3B9ai"
                    },
                    "href": "https://api.spotify.com/v1/artists/1Cs0zKBU1kc0i8ypK3B9ai",
                    "id": "1Cs0zKBU1kc0i8ypK3B9ai",
                    "name": "David Guetta",
                    "type": "artist",
                    "uri": "spotify:artist:1Cs0zKBU1kc0i8y

In [ ]:
def analyze_playlist_dict(playlist_dict):
  playlists = {}
  for i, (key, val) in enumerate(playlist_dict.items()):
    playlist = analyze_playlist(*val)
    print(len(playlist))
    playlists[key] = playlist
  return playlists

    

In [ ]:
playlist_dict = {
    'Top Tracks of 2018' : ("Spotify" , "37i9dQZF1DX1HUbZS4LEyL"),
    'Top Tracks of 2019' : ("Spotify" , "37i9dQZF1DX7fxmJCMXN72"),
    'Top Tracks of 2020' : ("Spotify" , "37i9dQZF1DX7Jl5KP2eZaS"),
    'Top Tracks of 2021' : ("Spotify" , "37i9dQZF1DX18jTM2l2fJY"),
    #Playlist of 10,122 songs by user DuckLover/F4F/Fox&WoldTeamAnim
    'Real THE LONGEST playlist ever' : ("DuckLover/F4F/Fox&WolfTeamAnim" , "1151nuJMpCvYjgPaN1HCRk")
}

#json_data = json.dumps(analyze_playlist_dict(playlist_dict), indent=4)
#print(json_data)

In [ ]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [ ]:
import csv

data = analyze_playlist_dict(playlist_dict)
data_file = open('/drive/My Drive/Colab Notebooks/data_file.csv', 'w')
csv_writer = csv.DictWriter(data_file, fieldnames=playlist_features_list)
csv_writer.writeheader()
 
count = 0
for playlist in playlist_dict:
    plData = data[playlist]
    for track in plData:
      csv_writer.writerow(track)
    count += 1
 
data_file.close()

Streaming output truncated to the last 5000 lines.
                "SN",
                "SR",
                "ST",
                "SV",
                "SZ",
                "TD",
                "TG",
                "TH",
                "TJ",
                "TL",
                "TN",
                "TO",
                "TR",
                "TT",
                "TV",
                "TW",
                "TZ",
                "UA",
                "UG",
                "US",
                "UY",
                "UZ",
                "VC",
                "VE",
                "VN",
                "VU",
                "WS",
                "ZA",
                "ZM"
            ],
            "external_urls": {
                "spotify": "https://open.spotify.com/album/3UpU3UWCvlk76ogUJzi40E"
            },
            "href": "https://api.spotify.com/v1/albums/3UpU3UWCvlk76ogUJzi40E",
            "id": "3UpU3UWCvlk76ogUJzi40E",
            "images": [
                {
  

In [ ]:
spotify_df = pd.read_csv('/drive/My Drive/Colab Notebooks/data_file.csv')
print(spotify_df)


                   artist  ... time_signature
0                   Drake  ...              4
1            XXXTENTACION  ...              4
2             Post Malone  ...              4
3             Post Malone  ...              4
4                   Drake  ...              4
...                   ...  ...            ...
10226                Kygo  ...              4
10227  Axwell /\ Ingrosso  ...              4
10228      Axel Johansson  ...              4
10229       Martin Garrix  ...              4
10230    Johannes Oerding  ...              4

[10231 rows x 19 columns]
